# http://www.op.gg/
# 최근 5개월 간 LOL play 시간 알아보기

<img src="op.png">

- 크롤링을 위한 selenium 패키지와
- 로딩시간을 위해 코드 실행을 잠시 멈추는 time 패키지 import

In [1]:
from selenium import webdriver
import time

In [2]:
# lol id를 인자로 받는 함수 생성
def lol_total_game_time(id):
    # op.gg 창 띄우기
    url = "http://www.op.gg/"
    driver = webdriver.Chrome()
    driver.get(url)
    
    # op.gg 메인 홈페이지 아이디 입력 창에 아이디 써넣기
    input_id = driver.find_element_by_css_selector("body > div.l-wrap.l-wrap--index > div.l-container > form > input")
    input_id.send_keys(id)
    
    # 아이디 제출 버튼 클릭하기
    button = driver.find_element_by_css_selector("body > div.l-wrap.l-wrap--index > div.l-container > form > button.summoner-search-form__button > i")
    button.click()
    
    # 페이지 로딩 시간 계산
    time.sleep(2)
    
    # 정보 업데이트를 위한 업데이트 버튼 클릭
#     update_button = driver.find_element_by_css_selector("#SummonerRefreshButton")
#     update_button.click()
#     time.sleep(3)

    # detail한 정보 가져올 때 필요한 detail 정보 보기 버튼 클릭
#     detail_button = driver.find_element_by_css_selector("#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameItemList > div:nth-child(1) > div > div.Content > div.StatsButton > div > div > a")
#     detail_button.click()
    
    
    # 게임 플레이 시간과 게임 한 날짜 데이터 프레임으로 만들기
    game_time = []
    game_day = []
    page_num = 3
    
    while page_num:
        try:
            for i in range(1,21):
                game_time.append(driver.find_element_by_css_selector("#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div:nth-child("+ str(page_num) +") > div:nth-child(" + str(i) +") > div > div.Content > div.GameStats > div.GameLength").text)
                game_day.append(driver.find_element_by_css_selector("#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div:nth-child("+ str(page_num) +") > div:nth-child(" + str(i) +") > div > div.Content > div.GameStats > div.TimeStamp > span").get_attribute("title"))
        except:
            break

        try: 
            driver.find_element_by_css_selector("#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameMoreButton.Box > a").click()
            page_num += 2
            time.sleep(3)
        except:    
             break
    
    df = pd.DataFrame(columns=["time", "day"])
    df["time"] = game_time
    df["day"] = game_day
    
    return df

### 아이디 입력 받아 데이터 프레임 형태로 저장

In [3]:
result = lol_total_game_time("갓분투")

In [4]:
result.shape

(654, 2)

In [5]:
for i in range(len(result)):
    result["time"][i] = result["time"][i].replace("m","")
    result["time"][i] = result["time"][i].replace("s","")

In [6]:
ls = []
for i in range(len(result)):
    m, s = result.time[i].split()
    ls.append(int(m) * 60 + int(s))
result["second"] = ls

In [7]:
result.tail()

,time,day,second
649,27 51,May 30 2018 7:23 AM,1671
650,22 55,May 30 2018 2:50 AM,1375
651,22 30,May 30 2018 2:21 AM,1350
652,31 9,May 30 2018 1:53 AM,1869
653,26 0,May 30 2018 1:16 AM,1560


In [8]:
total_seconds = result.second.sum()
total_seconds # 초

1087349

In [9]:
total_minutes = total_seconds / 60
total_minutes # 분

18122.483333333334

In [10]:
total_hours = total_minutes / 60
total_hours # 시간

302.0413888888889

In [11]:
total_days = total_hours / 24
total_days # 일

12.58505787037037

In [12]:
0.67 * 24

16.080000000000002

### 약 12일 16시간